# Testing

A notebook to test the implementation

In [1]:
import gzip
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


## 0 Preprocessing

For testing, we use the `MNIST` data. We first extract the data from `gzip` file.

In [2]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return (tf.cast(image, tf.float32)/255.0 , label)  

def preProcess(dset, batch_size):
  '''Pre-processes the dataset.'''
  dset = dset.map(normalize_img)
  dset = dset.shuffle(len(dset))
  dset = dset.batch(batch_size)
  return dset
  
# prepare data
def get_imgs(dir):
    with gzip.open(dir, 'r') as f:
        # first 4 bytes (some number)
        _ = int.from_bytes(f.read(4), 'big')
        # no. of images
        num_imgs = int.from_bytes(f.read(4), 'big')
        # row count
        row_cnt = int.from_bytes(f.read(4), 'big')
        # column count
        col_cnt = int.from_bytes(f.read(4), 'big')

        img_data = f.read()
        images = np.frombuffer(img_data, dtype=np.uint8).\
            reshape((num_imgs, row_cnt, col_cnt))
        return images

def get_labels(dir):
    with gzip.open(dir, 'r') as f:
        _ = int.from_bytes(f.read(4), 'big')
        label_cnt = int.from_bytes(f.read(4), 'big')
        print(label_cnt)
        label_data = f.read()
        labels = np.frombuffer(label_data, dtype=np.uint8)
        return labels

We now make `dataset`. Note that we used the `tf.data.Dataset.from_tensor_slices` to create dataset from a tuple of numpy arrays; if they were instead lists, then there will be an error

In [3]:
def getLocalDset(batch_size = 128):
    imgs     = get_imgs('data/train-images-idx3-ubyte.gz')
    labels   = get_labels('data/train-labels-idx1-ubyte.gz')
    split    = int(0.75*len(imgs))
    ds_train = tf.data.Dataset.from_tensor_slices((imgs[:split], labels[:split]))
    ds_test  = tf.data.Dataset.from_tensor_slices((imgs[split:], labels[split:]))
    print(len(ds_train))
    print(len(ds_test))
    return preProcess(ds_train, batch_size), preProcess(ds_test, batch_size)

We can also get the data by downloading

In [4]:
def fetch_img(ds, batch_size = 128):
  ds = ds.cache()
  # ds = ds.shuffle(ds_info.splits['train'].num_examples)
  ds = ds.batch(batch_size)
  # ds = ds.prefetch(tf.data.AUTOTUNE)
  return ds

def getRemoteDset(batch_size = 128, name = 'MNIST'):

  '''Prepares the MNIST dataset for training.'''  

  if name == 'MNIST':
    (ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    # split=['test', 'train[0%:10%]','train[10%:]'],
    split = ['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info = True
    )

  ds_train = ds_train.map(normalize_img, 
                          num_parallel_calls=tf.data.AUTOTUNE)
  ds_test = ds_test.map(normalize_img, 
                          num_parallel_calls=tf.data.AUTOTUNE)
  ds_train = fetch_img(ds_train, batch_size)
  ds_test = fetch_img(ds_test, batch_size)

  return (ds_train, ds_test), ds_info

(ds_train, ds_test), ds_info = getRemoteDset()

## 1 Running experiment

First define some hyperparameters

In [5]:
layers = [300, 100, 10] # use [784, 100, 10]

activation = 'relu'

lr = 0.001

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

metrics = tf.keras.metrics.SparseCategoricalAccuracy()    

train_loss = tf.keras.metrics.Mean

train_acc = tf.keras.metrics.SparseCategoricalAccuracy  

epochs = 1

num_pruning = 5  

step_perc = 0.5

In [7]:
from pruning import *
from models import *

model_params = {'layers': layers,
                'activation': 'relu',
                'BatchNorm':False,
                'Dropout':None,
                'optimizer': tf.keras.optimizers.Adam(lr),
                'loss_fn': loss_fn,
                'metrics': metrics
                }

train_params = {'train_loss': train_loss,
                'train_acc': train_acc
                }

prune_exp = pruning(ds_train, ds_test, model_params,
                    train_params,
                    epochs, num_pruning, step_perc)
                    
# prune_exp.test_training()

In [8]:
prune_exp.prune()


 
 Iterative pruning round: 0 
 


 
 After pruning, the total no. of nonzero weights is: 266200 
 


 Start original model training. 

Epoch 0 for original
OG train acc 0.9224333167076111
Original model accuracy 0.9603000283241272 


 Original model training finished. Highest Accuracy 0.9603000283241272 


 
 After pruning, the total no. of nonzero weights is: 133100 
 


 Start Lottery ticket training 

Epoch 0 for lottery ticket
Ticket train acc 0.8335000276565552 

Lottery ticket accuracy 0.9172000288963318 


 Lottery ticket training finished. Highest Accuracy 0.9172000288963318 

INFO:tensorflow:Assets written to: saved_models/ticket_acc_0.9172000288963318 pruning round_0\assets


INFO:tensorflow:Assets written to: saved_models/ticket_acc_0.9172000288963318 pruning round_0\assets



 
 Iterative pruning round: 1 
 


 
 After pruning, the total no. of nonzero weights is: 133100 
 


 Start original model training. 

Epoch 0 for original
OG train acc 0.9265333414077759
Original model accuracy 0.9628000259399414 


 Original model training finished. Highest Accuracy 0.9628000259399414 


 
 After pruning, the total no. of nonzero weights is: 67096 
 


 Start Lottery ticket training 

Epoch 0 for lottery ticket
Ticket train acc 0.881850004196167 

Lottery ticket accuracy 0.9323999881744385 


 Lottery ticket training finished. Highest Accuracy 0.9323999881744385 

INFO:tensorflow:Assets written to: saved_models/ticket_acc_0.9323999881744385 pruning round_1\assets


INFO:tensorflow:Assets written to: saved_models/ticket_acc_0.9323999881744385 pruning round_1\assets



 
 Iterative pruning round: 2 
 


 
 After pruning, the total no. of nonzero weights is: 67096 
 


 Start original model training. 

Epoch 0 for original
OG train acc 0.910183310508728
Original model accuracy 0.9506999850273132 


 Original model training finished. Highest Accuracy 0.9506999850273132 


 
 After pruning, the total no. of nonzero weights is: 35354 
 


 Start Lottery ticket training 

Epoch 0 for lottery ticket
Ticket train acc 0.8773666620254517 

Lottery ticket accuracy 0.9261000156402588 


 Lottery ticket training finished. Highest Accuracy 0.9261000156402588 

INFO:tensorflow:Assets written to: saved_models/ticket_acc_0.9261000156402588 pruning round_2\assets


INFO:tensorflow:Assets written to: saved_models/ticket_acc_0.9261000156402588 pruning round_2\assets



 
 Iterative pruning round: 3 
 


 
 After pruning, the total no. of nonzero weights is: 35354 
 


 Start original model training. 

Epoch 0 for original
OG train acc 0.8942000269889832
Original model accuracy 0.9391000270843506 


 Original model training finished. Highest Accuracy 0.9391000270843506 


 
 After pruning, the total no. of nonzero weights is: 19708 
 


 Start Lottery ticket training 

Epoch 0 for lottery ticket
Ticket train acc 0.8580499887466431 

Lottery ticket accuracy 0.9132999777793884 


 Lottery ticket training finished. Highest Accuracy 0.9132999777793884 

INFO:tensorflow:Assets written to: saved_models/ticket_acc_0.9132999777793884 pruning round_3\assets


INFO:tensorflow:Assets written to: saved_models/ticket_acc_0.9132999777793884 pruning round_3\assets



 
 Iterative pruning round: 4 
 


 
 After pruning, the total no. of nonzero weights is: 19708 
 


 Start original model training. 

Epoch 0 for original
OG train acc 0.8436166644096375
Original model accuracy 0.9175999760627747 


 Original model training finished. Highest Accuracy 0.9175999760627747 


 
 After pruning, the total no. of nonzero weights is: 11254 
 


 Start Lottery ticket training 

Epoch 0 for lottery ticket
Ticket train acc 0.7550333142280579 

Lottery ticket accuracy 0.8676999807357788 


 Lottery ticket training finished. Highest Accuracy 0.8676999807357788 

INFO:tensorflow:Assets written to: saved_models/ticket_acc_0.8676999807357788 pruning round_4\assets


INFO:tensorflow:Assets written to: saved_models/ticket_acc_0.8676999807357788 pruning round_4\assets


In [9]:
prune_exp.removeLogs()

Removing 20220806-130249
Removing 20220806-130748
Removing 20220806-131740


In [10]:
prune_exp.removeModels()

Removing ticket_acc_0.8676999807357788 pruning round_4
Removing ticket_acc_0.9132999777793884 pruning round_3
Removing ticket_acc_0.9172000288963318 pruning round_0
Removing ticket_acc_0.9261000156402588 pruning round_2
Removing ticket_acc_0.9323999881744385 pruning round_1
